# News Clustering using KMeans Algorithm
By Datetime : 2016-08-29 ~ 2016-09-05

In [1]:
import numpy as np
import pandas as pd

## Load data from Pickle 

In [2]:
train = pd.read_pickle("../datastore/clustering.p")
train = train.reset_index(drop=True)

In [3]:
headline = pd.read_pickle("../datastore/headline.p")
headlines = headline['headline'].tolist()

In [4]:
headlines

['지표로 보는 경제',
 '오늘의',
 '화제의 분양현장',
 '뉴스룸',
 '카드뉴스',
 '조선일보',
 'TV조선',
 '특집',
 '이슈',
 'Weekly BIZ',
 '금주의',
 '사회공헌 Together',
 '신나는 공부',
 '포토',
 '리빙포인트',
 '경제계 인사']

## Select Categories
- 포함 : 경제, 문화, 건강, 과학, 사회, 정치, 스포츠
- 제외 : 종합, 정보없음, 인물, 사설

In [5]:
categories = ['경제', '과학', '사회', '정치']

In [6]:
economy = train[train['name']=='경제']
science = train[train['name']=='과학']
social = train[train['name']=='사회']
politics = train[train['name']=='정치']

## Preprocessing
1. Datetime (16-09-11 ~ 16-09-17)
2. Remove stopwords (regex, hanja)
3. POS Tagging with KoNLPy, Mecab
4. Using bigram

In [7]:
import datetime
from konlpy.tag import Mecab
import hanja
import re
pd.options.mode.chained_assignment = None  # default='warn'

In [8]:
mecab = Mecab()

In [9]:
def text_cleaning(text):
    text = hanja.translate(text, 'substitution')
    text = re.sub('[^가-힝0-9a-zA-Z\\s]', ' ', text)
    for headline in headlines:
        text = text.replace(headline, ' ')
    return text

In [10]:
def tokenize(data):
    return [' '.join(e for e in mecab.nouns(data))]

In [11]:
economy['title_flat'] = economy['title'].apply(lambda text: text_cleaning(text))
science['title_flat'] = science['title'].apply(lambda text: text_cleaning(text))
social['title_flat'] = social['title'].apply(lambda text: text_cleaning(text))
politics['title_flat'] = politics['title'].apply(lambda text: text_cleaning(text))
title_economy = [tokenize(each[1]['title_flat']) for each in economy.iterrows()]
title_science = [tokenize(each[1]['title_flat']) for each in science.iterrows()]
title_social = [tokenize(each[1]['title_flat']) for each in social.iterrows()]
title_politics = [tokenize(each[1]['title_flat']) for each in politics.iterrows()]

## Training
1. Feature extraction - TfidVectorizer
2. Decomposition - PCA
3. Cluster - KMeans

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [13]:
vectorizer = TfidfVectorizer(lowercase=False, ngram_range=(1,2))
title_economy_flat = [item for sublist in title_economy for item in sublist]
title_science_flat = [item for sublist in title_science for item in sublist]
title_social_flat = [item for sublist in title_social for item in sublist]
title_politics_flat = [item for sublist in title_politics for item in sublist]
x_list_economy = vectorizer.fit_transform(title_economy_flat)
x_list_science = vectorizer.fit_transform(title_science_flat)
x_list_social = vectorizer.fit_transform(title_social_flat)
x_list_politics = vectorizer.fit_transform(title_politics_flat)

In [14]:
x_list_economy = PCA(n_components=10).fit_transform(x_list_economy.toarray())
x_list_science = PCA(n_components=10).fit_transform(x_list_science.toarray())
x_list_social = PCA(n_components=10).fit_transform(x_list_social.toarray())
x_list_politics = PCA(n_components=10).fit_transform(x_list_politics.toarray())

### Scoring

In [15]:
from sklearn.metrics import silhouette_samples, silhouette_score
from IPython.display import display, HTML
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

### Best Silhoutte Score

In [42]:
best_score = 0.0
best_k = 0

In [43]:
for k in range(3, 15):
    km = KMeans(n_clusters=k, n_jobs=-1).fit(x_list_politics)
    score = silhouette_score(x_list_politics, km.labels_)
    if best_score < score:
        best_score = score
        best_k = k
print("In Clusters =", best_k, ", Best score is : %0.3f" % best_score)

In Clusters = 3 , Best score is : 0.560


### K-Means Algorithm

In [44]:
km = KMeans(n_clusters=best_k, n_jobs=-1).fit(x_list_politics)
labels = km.labels_
centroids = km.cluster_centers_
print(km.inertia_)

21.8639400152


In [45]:
politics['cluster'] = labels

## Choose Best Cluster
1. Cluster size < 500
2. Recent published
3. Minimum inertia

### Compare best cluster

In [47]:
sample_silhouette_values = silhouette_samples(x_list_politics, labels)
sample_silhouette_score = []
best_cluster = []
cluster_num = best_k

for i in range(cluster_num):
    ith_cluster_silhouette_values = \
        sample_silhouette_values[labels == i]
        
    print('Cluster %d: ' % (i), abs(ith_cluster_silhouette_values.mean()))
    sample_silhouette_score.append(abs(ith_cluster_silhouette_values.mean()))

sample_silhouette_score.sort(reverse=True)
sample_silhouette_score = sample_silhouette_score[:3]

Cluster 0:  0.111120746772
Cluster 1:  0.576164051419
Cluster 2:  0.703484791879


In [48]:
for i in range(cluster_num):
    ith_cluster_silhouette_values = \
        sample_silhouette_values[labels == i]
        
    if abs(ith_cluster_silhouette_values.mean()) in sample_silhouette_score:
        best_cluster.append(i)

In [49]:
politics = politics[politics['cluster'].isin(best_cluster)]
politics.cluster.unique()

array([1, 2, 0])

## Result

In [23]:
cluster_data = []

for cluster_index in range(cluster_num):
    if cluster_index in best_cluster:
        cluster_data.append(economy[economy['cluster'] == cluster_index])
    
for i, d in enumerate(cluster_data):
    print('Cluster %d:' % (i), 'Size %d' % (len(d)))

    display(d[['title', 'category']].sample(min(10, len(d))))
    print('\n\n')

Cluster 0: Size 484


,title,category
1406,[알립니다]‘신기후체제 미래 에너지’ 세미나에 초대합니다,뉴스 > 경제 > 경제일반
2236,폭염에 채소값 폭등…배추값 60% 올랐네,경제 > 경제일반
1389,“한국 여성 노동참여율 亞太 16개국중 12위”,뉴스 > 경제 > 경제일반
1395,"[간추린 뉴스/ 단신]연암대, 8일부터 수시 1차 신입생 모집",뉴스 > 경제 > 경제일반
1445,[간추린 뉴스/ 단신]추석연휴 전국 800개 문화시설 무료 개방 外,뉴스 > 경제 > 경제일반
2280,‘1등 선사’ 침몰 위기…수출입 물류 비상·부산 경제 ‘직격탄’,경제 > 경제일반
1466,"“노후 준비 양극화… 저소득층 사적연금 확대, 정부가 나서야”",뉴스 > 경제 > 경제일반
1488,[분양 정보]서울 봉천동 ‘서울대역 힐링스테이트’ 2400채 外,뉴스 > 경제 > 경제일반
3061,동남아 최대 자동차 생산국으로 부상한 태국,경제 > 자동차
764,"ETF 신탁 운용 10년 노하우, 다양한 상품 선봬",경제 > 유통ㆍ소비자





Cluster 1: Size 8


,title,category
2239,한진해운 회생절차 개시…산은·현대상선은 청산 대비 TF,경제 > 경제일반
9,한진 美·유럽 노선에 현대상선 선박 13척 투입,경제
2268,"정부, 현대상선이 한진해운 우량 자산 인수토록 한다",경제 > 경제일반
2227,현대상선 새 CEO 후보자에 유창근 인천항만공사 사장,경제 > 경제일반
2232,현대상선 새 CEO 후보자에 유창근 인천항만공사 사장,경제 > 경제일반
1167,유창근 2년만에 컴백 현대상선 사장에 내정,뉴스 > 경제 > 기업
1176,"2위 현대상선, 1위 한진해운 흡수하나",뉴스 > 경제 > 기업
24,"""파산위기 한진해운, 현대상선과 합병해야 생존""",경제





Cluster 2: Size 5


,title,category
2187,미 고용지표 부진…9월 금리인상 무산?,경제 > 금융·재테크
1452,美 금리인상 우려에… 韓日 통화스와프 논의 재개,뉴스 > 경제 > 경제일반
2292,이주열 총재 “미 연준 금리 인상 가능성 높아져”,경제 > 경제일반
2189,미 고용지표 부진…“9월 금리인상 물 건너 가나?”,경제 > 금융·재테크
1500,美 금리인상 신호에 달러 강세… 환율 11.3원 급등,뉴스 > 경제 > 금융


In [33]:
cluster_data = []

for cluster_index in range(cluster_num):
    if cluster_index in best_cluster:
        cluster_data.append(science[science['cluster'] == cluster_index])
    
for i, d in enumerate(cluster_data):
    print('Cluster %d:' % (i), 'Size %d' % (len(d)))

    display(d[['title', 'category']].sample(min(10, len(d))))
    print('\n\n')

Cluster 0: Size 2


,title,category
2807,“파이로는 핵쓰레기 청소부가 아니라 돈 먹는 하마”,과학 > 과학일반
2075,홍대 ‘클럽데이’의 끝은 쓰레기 천지,뉴스 > 사회 > 환경





Cluster 1: Size 2


,title,category
2087,"‘테니스 엘보’ 환자 10명 중 6명이 40, 50대",뉴스 > IT/의학 > 건강
2093,국내 11번째 지카 바이러스 환자 발생,뉴스 > IT/의학 > 건강





Cluster 2: Size 1


,title,category
2802,인류 고유의 최고 발명품은 단위,과학 > 과학일반





Cluster 3: Size 1


,title,category
2091,한림대 강남성심병원 두통 건강강좌,뉴스 > IT/의학 > 건강





Cluster 4: Size 1


,title,category
1740,"첨단보안기술, 홍채-정맥이어 얼굴-몸짓 인식까지",뉴스 > IT/의학 > 정보/과학일반


In [41]:
cluster_data = []

for cluster_index in range(cluster_num):
    if cluster_index in best_cluster:
        cluster_data.append(social[social['cluster'] == cluster_index])
    
for i, d in enumerate(cluster_data):
    print('Cluster %d:' % (i), 'Size %d' % (len(d)))

    display(d[['title', 'category']].sample(min(10, len(d))))
    print('\n\n')

Cluster 0: Size 622


,title,category
2616,"화성시, 우 수석 처가 ‘차명 땅’ 자료 제출...검찰 미술품 추적",사회 > 사회일반
1123,"안산 시화쓰레기매립지, 수도권 최대 정원 탈바꿈",뉴스 > 사회 > 사회일반
1135,[@뉴스룸/김재영]‘요요’는 다시 온다,뉴스 > 사회 > 사회일반
2858,[경기도청소년기자단] 경기도는 지금,사회 > 교육
1092,석촌호수에 내려앉은 ‘슈퍼문’,뉴스 > 사회 > 사회일반
2013,[신나는 공부]“경쟁자에게도 박수 보내요”,뉴스 > 사회 > 교육
2480,한라산 구상나무숲 복원 본격 추진한다,사회 > 전국
2592,“‘에어포켓’ 공기주입 효과 없었다…구조 시간만 낭비”,사회 > 사회일반
2905,[숙명여자대학교] 프라임사업 선정으로 공과대학 개편,사회 > 교육
2991,세월호·백남기·사드 ‘언론 피해자’ 증언대 서다,사회 > 미디어





Cluster 1: Size 9


,title,category
2575,정운호한테 뇌물받은 부장판사 구속...대법원 “참담하다”,사회 > 사회일반
1081,충격의 사법부… 정운호에 억대수수 혐의 현직 부장판사 영장,뉴스 > 사회 > 사회일반
2604,‘레인지로버 판사’ 구속영장 청구,사회 > 사회일반
2621,‘정운호 금품 수수 의혹’ 현직 부장판사 긴급체포,사회 > 사회일반
1162,"檢, ‘공짜 외제차’ 부장판사 구속영장 방침",뉴스 > 사회 > 사건·범죄
2644,‘정운호 금품수수’ 의혹 현직 부장판사 소환 조사,사회 > 사회일반
1635,[사설]‘정운호 뇌물’ 부장판사 구속…대한민국 法治가 무너진다,뉴스 > 사회 > 검찰-법원판결
1161,‘정운호 금품’ 받은 부장판사 구속,뉴스 > 사회 > 사건·범죄
1112,‘공짜 외제차’ 부장판사 9월 첫째 주내 소환,뉴스 > 사회 > 사회일반





Cluster 2: Size 7


,title,category
473,"본사, 송희영 주필 보직 해임",사회
2690,‘초호화 외유’ 의혹 송희영 조선일보 주필 사의표명,사회 > 사회일반
2992,송희영 주필 사의 표명…조선일보 보직해임 결정,사회 > 미디어
2986,"조선일보, 송희영 전 주필 관련 의혹에 지면 통해 “사과”",사회 > 미디어
2632,"검찰, 송희영 전 조선일보 주필 수사 본격화",사회 > 사회일반
1107,"조선일보, 호화외유 의혹 송희영 前주필 사표 수리",뉴스 > 사회 > 사회일반
2658,"조선일보, 송희영 전 주필 사표 수리",사회 > 사회일반





Cluster 3: Size 4


,title,category
2494,이청연 인천 교육감 영장 기각,사회 > 전국
2520,이청연 인천교육감 오늘 영장실질심사,사회 > 전국
2404,이청연 인천교육감 영장 기각되자 정치자금법 수사,사회 > 전국
2466,이청연 인천교육감 “시민들에게 죄송” 사과,사회 > 전국





Cluster 4: Size 3


,title,category
2686,"대법, 상주 ‘농약사이다’ 할머니 무기징역 확정",사회 > 사회일반
2684,‘상주 농약 사이다’ 주범 할머니 무기징역 확정,사회 > 사회일반
1638,대법 ‘상주 농약사이다’ 할머니 무기징역 확정,뉴스 > 사회 > 검찰-법원판결


In [50]:
cluster_data = []

for cluster_index in range(cluster_num):
    if cluster_index in best_cluster:
        cluster_data.append(politics[politics['cluster'] == cluster_index])
    
for i, d in enumerate(cluster_data):
    print('Cluster %d:' % (i), 'Size %d' % (len(d)))

    display(d[['title', 'category']].sample(min(10, len(d))))
    print('\n\n')

Cluster 0: Size 19


,title,category
2169,여야 추경안 극적 타결…오늘 본회의 열어 처리키로,정치 > 정치일반
981,"20代국회 법안발의 1795건, 통과는 0건",뉴스 > 정치 > 국회
2132,국회의장 개회사에 ‘집단퇴장’…26년만에 처음,정치 > 국회·정당
1024,"[사설]20대 첫 정기국회 파행, 이젠 丁의장까지 민생에 재 뿌리나",뉴스 > 정치 > 정치일반
556,"정세균 '우병우·사드' 발언… 與, 정기국회 첫날부터 보이콧",정치
559,"與野, 오늘 본회의서 추경 처리키로",정치
2124,‘새누리당의 강경투쟁’ 속에서도 막후접촉…추경안 처리 도출,정치 > 국회·정당
2119,정기국회 ‘전운’ 고조… 노동·검찰개혁 등 곳곳 ‘지뢰’,정치 > 국회·정당
2178,정진석 “야당 추경 처리 않으면 구조조정·백남기 청문회 안돼”,정치 > 정치일반
2129,국회의장이 ‘우병우’ 비판했다고…추경까지 미룬 새누리,정치 > 국회·정당





Cluster 1: Size 455


,title,category
2924,박 대통령 “북핵 위협 제거되면 사드 필요성도 없어질 것”,정치 > 국방·북한
496,"일 정부, 10억엔 송금…위안부 할머니들 우비 입고 반대 목청",정치 > 외교
2930,새누리 “핵잠수함 도입”…한반도 비핵화 원칙 흔들어,정치 > 국방·북한
2143,남경필 “모병제로 청년일자리 창출”,정치 > 국회·정당
2927,‘9월의 독립운동가’에 청산리대첩 공훈 나중소 선생,정치 > 국방·북한
2117,"추미애, 박정희·이승만 묘역 참배한 뒤 박 대통령에게…",정치 > 정치BAR
1040,檢 ‘우병우 아들 경찰상관’ 통화기록 조회… 보직청탁 의혹 추적,뉴스 > 정치 > 정치일반
1019,경북-전남 ‘中企 살리기-일자리 만들기’ 손잡아,뉴스 > 정치 > 정치일반
561,"""닥치세요"" ""멍텅구리""… 막말 오간 趙청문회",정치
2359,두테르테 ‘초법적 살인’에 맞선 강심장 의원,국제 > 아시아·태평양





Cluster 2: Size 7


,title,category
492,한중·한미 정상회담 잇달아 열릴 듯,정치 > 외교
490,朴대통령·시진핑 '사드 정상회담',정치 > 외교
1002,"朴대통령, 러-中-美와 연쇄 정상회담",뉴스 > 정치 > 정치일반
507,"박 대통령, 러시아·중국·미국 연쇄 정상회담",정치 > 청와대
497,9월4~5일 항저우 G20 정상회의 때 한-중 정상회담 열릴 듯,정치 > 외교
2350,미·중 3일 정상회담…사드 평행선 달릴 듯,국제 > 미국·중남미
503,"박 대통령, 5일 오전 시진핑 주석과 한-중 정상회담",정치 > 청와대
